# Grasple Test to Microsoft Excel

o,porty Below you can upload a `.csv`-file exported from a Grasple test. The file will be converted to a Microsoft Excel file, which downloads automatically after generation.

In [ ]:
import micropip
await micropip.install("dash")

import dash
from dash import dcc, html, Input, Output, State, dash_table
import pandas as pd
import base64
import io

# Initialiseer de Dash app
app = dash.Dash(__name__)

# Layout van de app
app.layout = html.Div([
    html.H2("CSV Upload en Weergave"),
    dcc.Upload(
        id='upload-data',
        children=html.Button('Upload CSV'),
        multiple=False
    ),
    html.Div(id='output-table')
])

# Callback voor het verwerken van de upload
@app.callback(
    Output('output-table', 'children'),
    Input('upload-data', 'contents'),
    State('upload-data', 'filename')
)
def update_table(contents, filename):
    if contents is None:
        return html.P("Upload een CSV-bestand om de inhoud te bekijken.")
    
    # Decodeer de inhoud van het bestand
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    try:
        df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
    except Exception as e:
        return html.P(f"Fout bij inlezen bestand: {e}")
    
    # Toon DataFrame als Dash DataTable
    return dash_table.DataTable(
        data=df.to_dict('records'),
        columns=[{'name': i, 'id': i} for i in df.columns],
        page_size=10,  # aantal rijen per pagina
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'}
    )

# Start de server
if __name__ == '__main__':
    app.run(debug=True)